In [ ]:
from django.http.response import Http404, HttpResponseRedirect
from django.urls import reverse
from django.shortcuts import render, get_list_or_404, get_object_or_404
from django.views.decorators.clickjacking import xframe_options_sameorigin
from django.http import HttpResponse

import os
import sys
import django
from user.models import User

from django.utils import timezone
import datetime
import json
import random
import string

os.chdir('/home/wxy/prj/Tostada')
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [ ]:
from tool.models import AuthorizationLevel

code

In [ ]:
(User.objects.get()).__class__.__name__